
</br>

# **Métodos de Financiamento Atuarial**
----
"_Sem dados, você é apenas mais uma pessoa com uma opinião_" - **W. E. Deming**

</br>

>[Marcos F Silva](https://audinalytics.netlify.app/)   
**Última Atualização:** 31/07/2022  
**Status:** em elaboração

In [ ]:
#@title
# #@title **Selecione a UF e o ANO desejados e presssione CTRL + F9**

options(scipen = 999,  message = FALSE, warn = -1)

# Define as escolhas possíveis
#sigla_uf = "RJ"  #@param ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']
#ano = "2022" #@param [2020, 2021, 2022]

# Instala os pacotes necessários
remotes::install_github("marcosfs2006/ADPrev", quiet = TRUE)
install.packages("writexl")

# Carrega os pacotes necessários------------------------------------------------
suppressPackageStartupMessages({
  library(ADPrev)
  library(httr)
  library(purrr)
  library(lubridate)
  library(stringr)
  library(tidyr)
  library(dplyr)
  library(writexl)
})

set_config(config(ssl_verifypeer = 0L))

# Definir uma função para pegar os dados do "DAIR_IDENTIFICACAO"----------------
# Esta função não está ainda no pacote ADPrev

get_draa_plano_beneficio <- function(...){

flag <- TRUE
    pagina = 0
    consulta <- list(...)
    dados_draa_plano_beneficio <- data.frame()
    while (flag) {
        draa_plano_beneficio <- httr::GET("https://apicadprev.economia.gov.br/DRAA_PLANO_BENEFICIO?", 
            query = append(consulta, list(offset = pagina)))
        httr::stop_for_status(draa_plano_beneficio, task = "Connect to the server! Try again later.")
        draa_plano_beneficio_txt <- httr::content(draa_plano_beneficio, as = "text", 
            encoding = "UTF-8")
        draa_plano_beneficio_json <- jsonlite::fromJSON(draa_plano_beneficio_txt, 
            flatten = FALSE)
        draa_plano_beneficio_df <- as.data.frame(draa_plano_beneficio_json[["results"]][["data"]])
        dados_draa_plano_beneficio <- dplyr::bind_rows(dados_draa_plano_beneficio, 
            draa_plano_beneficio_df)
        flag <- draa_plano_beneficio_json[["results"]][["rowLimitExceeded"]]
        pagina <- pagina + 1
        Sys.sleep(1)
    }
    return(dados_draa_plano_beneficio)
}


# Faz a extração dos dados
#draa <- get_draa_plano_beneficio(sg_uf = sigla_uf, dt_exercicio = ano)


#===PEGAR TODOS OS ESTADOS======================================================

sigla_uf <- c('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
#draa <- get_draa_plano_beneficio(sg_uf = sigla_uf, dt_exercicio = ano)

draa_plano_beneficio_2022 <- map_df(sigla_uf, ~get_draa_plano_beneficio(sg_uf = .x, dt_exercicio = 2022)) 
#draa_plano_beneficio_2021 <- map_df(sigla_uf, ~get_draa_plano_beneficio(sg_uf = .x, dt_exercicio = 2021)) 
#draa_plano_beneficio_2020 <- map_df(sigla_uf, ~get_draa_plano_beneficio(sg_uf = .x, dt_exercicio = 2020))

# Gera o arquivo com a base de dados bruta
write_xlsx(draa_plano_beneficio_2022, "draa_plano_beneficio_2022.xlsx")
#write_xlsx(draa2021, "draa2021.xlsx")
#write_xlsx(draa2020, "draa2020.xlsx")

#save(draa2022, file="draa2022.RData")
#save(draa2021, file="draa2021.RData")
#save(draa2020, file="draa2020.RData")



Installing 2 packages: pillar, tibble

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
draa_previdenciario <- draa_plano_beneficio_2022 %>% 
   filter(tp_plano == "Previdenciário",
          nr_plano_benefico == 110102) %>%
   mutate(dt_envio = ymd(str_sub(dt_envio, 1, 10))) %>%
   group_by(nr_cnpj_entidade) %>%
   filter(dt_envio == max(dt_envio)) %>%
   ungroup() %>%
   distinct(nr_cnpj_entidade, .keep_all = TRUE)


In [ ]:
# Verificar se existem RPPS em duplicidade após o tratamento de dados
draa_previdenciario %>% 
  group_by(nr_cnpj_entidade) %>%
  filter(n() > 1) %>%
  arrange(no_ente)

nr_cnpj_entidade,sg_uf,no_ente,dt_exercicio,dt_envio,te_situacao,tp_plano,tp_massa,no_tipo_norma,nr_norma_fundamento,dt_norma_fundamento,te_dispositivo_norma,nr_plano_benefico,no_beneficio_plano,nr_regime_financeiro,ds_regime_financeiro,ds_metodo_financiamento
<chr>,<chr>,<chr>,<int>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>


In [ ]:
# Distribuição de frequências dos métodos de financiamento 
draa_previdenciario %>% 
   count(ds_regime_financeiro) %>%
   arrange(desc(n))

ds_regime_financeiro,n
<chr>,<int>
REGIME FINANCEIRO DE CAPITALIZAÇÃO - CRÉDITO UNITÁRIO PROJETADO,702
REGIME FINANCEIRO DE CAPITALIZAÇÃO - AGREGADO,420
REGIME FINANCEIRO DE CAPITALIZAÇÃO - IDADE DE ENTRADA NORMAL,214
REGIME FINANCEIRO DE CAPITALIZAÇÃO - PRÊMIO NIVELADO INDIVIDUAL,159
REGIME FINANCEIRO DE CAPITALIZAÇÃO - OUTROS,28
REGIME FINANCEIRO DE REPARTIÇÃO DE CAPITAIS DE COBERTURA,1


In [ ]:
# exportar a base de dados tratada
write_xlsx(draa_previdenciario, "base_tratada_2022.xlsx")  